In [1]:
import numpy as np
import pandas as pd
import math
import requests 
import matplotlib as plt
import seaborn as sns
import yfinance as yf
import pandas_datareader as web
from pandas_datareader import data
from bs4 import BeautifulSoup as bs
from scipy import stats
import sqlite3

#Create db
conn = sqlite3.connect('financial_data.db')

# Create a cursor
cursor = conn.cursor()
"""
# Create the first table
all_time_prices_query = '''
CREATE TABLE all_time_prices (
    Date INTEGER PRIMARY KEY,
    Open FLOAT,
    High FLOAT, 
    Low FLOAT, 
    Close FLOAT, 
    Volume FLOAT,
    Dividends FLOAT, 
    Stock Splits FLOAT, 
    Ticker TEXT NOT NULL    
);
'''

cursor.execute(all_time_prices_query)

"""

momentum_query = '''
CREATE TABLE momentum (
    Date INTEGER PRIMARY KEY,
    Open FLOAT,
    High FLOAT, 
    Low FLOAT, 
    Close FLOAT, 
    Volume FLOAT,
    Dividends FLOAT, 
    Stock Splits FLOAT, 
    Ticker TEXT NOT NULL    
);
'''

cursor.execute(momentum_query)

conn.commit()

# Close the connection
conn.close()


NameError: name 'OperationalError' is not defined

In [2]:
#calulate the Hurst exponent of a stock
def get_hurst_exponent(time_series, max_lag):
    """Returns the Hurst Exponent of the time series"""
        
    lags = range(2, max_lag)

    # variances of the lagged differences
    tau = [np.std(np.subtract(time_series[lag:], time_series[:-lag])) for lag in lags]

    # calculate the slope of the log plot -> the Hurst Exponent
    reg = np.polyfit(np.log(lags), np.log(tau), 1)

    return reg[0]

#Create first table with all stock info from yfinance
tick = pd.read_csv("all_stocks")
all_time_cols = ['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Dividends', 'Stock Splits', 'Ticker']
momentum_cols = ['Ticker','Price',
                '1y Price Return','1y percentile', 
                '6m Price Return', '6m percentile', 
                '3m Price Return', '3m percentile',
                '1m Price Return','1m percentile',
                'Weighted Hurst Exponent']
momentum = pd.DataFrame(columns = momentum_cols)
all_time = pd.DataFrame(columns = all_time_cols)
count = 0 
for i in range(len(tick)): 
    try:
        #Creating the df to be added to all_time_prices
        ticker = tick["0"][i]
        info = yf.Ticker(ticker).history(period='max')
        info.reset_index(inplace = True)
        info['Ticker'] = ticker

        # Append the first dataframe to the table
        all_time = pd.concat([all_time, info])

        #Creating the df to be added to momentum
        y = info.truncate(before=pd.Timestamp('today') - pd.DateOffset(months=12))
        m6 = info.truncate(before=pd.Timestamp('today') - pd.DateOffset(months=6))
        m3 = info.truncate(before=pd.Timestamp('today') - pd.DateOffset(months=3))
        m1 = info.truncate(before=pd.Timestamp('today') - pd.DateOffset(months=1))

        momentum['Ticker'] = tick['0'][i]  
        try: 
            perc_change1y = ((y.iloc[-1]['Close'] - y.iloc[0]['Close'])/y.iloc[0]["Close"])
            d['1y Price Return'] = perc_change1y
            perc_change6m = ((m6.iloc[-1]['Close'] - m6.iloc[0]['Close'])/m6.iloc[0]["Close"])
            d['6m Price Return'] = perc_change6m
            perc_change3m = ((m3.iloc[-1]['Close'] - m3.iloc[0]['Close'])/m3.iloc[0]["Close"])
            d['3m Price Return'] = perc_change3m
            perc_change1m = ((m1.iloc[-1]['Close'] - m1.iloc[0]['Close'])/m1.iloc[0]["Close"])
            d['1m Price Return'] = perc_change1m

            d['Price'] = d['Close'] 
            d['1y percentile'] = 0
            d['6m percentile'] = 0
            d['3m percentile'] = 0
            d['1m percentile'] = 0
            d['Weighted Hurst Exponent'] = 0
            d = d.iloc[-1]
        except IndexError: 
            print('Index Error when producing momentum table')
            continue

    except TypeError: 
        print("Nonetype found for: " + tick["0"][i])
        continue
    except IndexError: 
        print("Couldn't find: ",tick["0"][i])
        continue
    except KeyError:
        print("Couldnt find key for: " + tick['0'][i])
        continue
    

#all_time.to_sql('all_time_prices', conn, if_exists='append', index=False)




In [3]:
all_time

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits,Ticker
0,1980-12-12 00:00:00-05:00,0.099874,0.100308,0.099874,0.099874,469033600,0.0,0.0,AAPL
1,1980-12-15 00:00:00-05:00,0.095098,0.095098,0.094663,0.094663,175884800,0.0,0.0,AAPL
2,1980-12-16 00:00:00-05:00,0.088149,0.088149,0.087715,0.087715,105728000,0.0,0.0,AAPL
3,1980-12-17 00:00:00-05:00,0.089886,0.090320,0.089886,0.089886,86441600,0.0,0.0,AAPL
4,1980-12-18 00:00:00-05:00,0.092492,0.092927,0.092492,0.092492,73449600,0.0,0.0,AAPL
...,...,...,...,...,...,...,...,...,...
448,2023-01-04 00:00:00-05:00,10.120000,10.120000,10.100000,10.100000,1529100,0.0,0.0,AAC
449,2023-01-05 00:00:00-05:00,10.120000,10.120000,10.110000,10.110000,121700,0.0,0.0,AAC
450,2023-01-06 00:00:00-05:00,10.120000,10.120000,10.110000,10.110000,90600,0.0,0.0,AAC
451,2023-01-09 00:00:00-05:00,10.110000,10.130000,10.110000,10.130000,1482200,0.0,0.0,AAC
